In [2]:
'''
M1. Integradora


Authors:
Luis Alberto Alcántara Cabrales A01634185
Alexa Serrano Negrete A01654063
Renet de Jesús Pérez Gómez A01640555
Vicente Javier Viera Guízar A01639784

Date:
11/17/2022
'''

#Model
import agentpy as ap
import random


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [3]:
parameters = {
    'K': 10,
    'citySize': 60,
    'streetSize': 4,
    'steps': 250,
}

In [16]:
class MyModel(ap.Model):
    def setup(self):
        #Create agents
        self.city = ap.AgentList(self, int(self.p.citySize ** 2))
        self.cars = ap.AgentList(self, self.p.K)
        
        #Create grid
        self.area = ap.Grid(self, [self.p.citySize] * 2)
        
        #Determinate the initial position of the cars
        carsPosition = initialPosition(self.cars, self.p.streetSize, self.p.citySize)
        
        #Add agents to enviroment
        self.area.add_agents(self.cars, carsPosition)
        
        
        #Defining types of agents
        self.city.agent_type = 0
        self.cars.agent_type = 1
        
    def step(self):
        cars = self.cars
        typePosition = 1
        for car in self.cars:
            if(typePosition==1):
                self.area.move_by(car, (1,0))

            if(typePosition==2):
                self.area.move_by(car, (-1,0))

            if(typePosition==3):
                self.area.move_by(car, (0,1))

            if(typePosition==4):
                self.area.move_by(car, (0,-1))
                typePosition = 1

            typePosition += 1

    def initialPosition(numCars, street, citySize):
        position = []
        '''
        If modulus is equal to 0, then it going to be in horizontal way
        if not, then it going to be in the vertical way 
        '''

        #Defining the street coordinates
        rightSide = int(((citySize - street) / 2) + 2)
        leftSide = int(rightSide + 1)
        endEnviroment = int(citySize - 1)


        typePosition = 1
        #Create initial positions for the car
        for i in range(len(numCars)):        
            if(typePosition==1):
                position.append((0, rightSide))

            if(typePosition==2):
                position.append((endEnviroment, leftSide))

            if(typePosition==3):
                position.append((rightSide, 0))

            if(typePosition==4):
                position.append((leftSide, endEnviroment))
                typePosition = 1

            typePosition += 1
        return position


In [17]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('agent_type')
    color_dict = {0:'#000', 1:'#c1c1c1', None:'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic Simulation\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))